In [1]:
import numpy as np
import pandas as pd
import altair as alt

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

In [2]:
#Question 1
np.random.seed(2025)

In [3]:
data = pd.read_csv('data/abalone.csv')

In [4]:
data.head()

Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  Viscera_weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell_weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7

In [5]:
data['Sex'].value_counts()

Sex
M    1528
I    1342
F    1307
Name: count, dtype: int64

In [6]:
#Question 2 
# remove infant observations
df = data[(data['Sex'] == 'M' )|( data['Sex'] == 'F')]
df.head()

Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  Viscera_weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
6   F   0.530     0.415   0.150        0.7775          0.2370          0.1415   

   Shell_weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
6         0.330     20

In [7]:
# check whether or not df have missing value or NaN
df.isnull().sum()

Sex               0
Length            0
Diameter          0
Height            0
Whole_weight      0
Shucked_weight    0
Viscera_weight    0
Shell_weight      0
Rings             0
dtype: int64

In [8]:
df['Sex'] = df['Sex'].replace({
    'M': 'Male',
    'F': 'Female'
})

/tmp/ipykernel_64835/1005458075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'] = df['Sex'].replace({


In [9]:
df['Sex'].value_counts()

Sex
Male      1528
Female    1307
Name: count, dtype: int64

In [10]:
plot_2 = alt.Chart(df).mark_bar().encode(
    x = alt.X('Sex'),
    y = alt.Y('count()')
)
plot_2 = plot_2.properties(width=300)
plot_2

alt.Chart(...)

In [11]:
df['Sex'].value_counts(normalize=True)

Sex
Male      0.538977
Female    0.461023
Name: proportion, dtype: float64

In [12]:
#Question 3
# split the data into test and training data (0.25:0.75)
train_df, test_df = train_test_split(df,test_size=0.25,stratify=df['Sex'])

In [13]:
train_df['Sex'].value_counts(normalize=True),test_df['Sex'].value_counts(normalize=True)

(Sex
 Male      0.53904
 Female    0.46096
 Name: proportion, dtype: float64,
 Sex
 Male      0.538787
 Female    0.461213
 Name: proportion, dtype: float64)

In [14]:
model = KNeighborsClassifier()
preprocessor = make_column_transformer(
    (StandardScaler(),['Length','Diameter','Height','Whole_weight','Shucked_weight','Viscera_weight','Shell_weight','Rings'])
)
knn_pipeline = make_pipeline(preprocessor,model)

In [15]:
param_grid = {"kneighborsclassifier__n_neighbors": range(1, 100,5)}

knn_grid = GridSearchCV(
    estimator = knn_pipeline,
    param_grid = param_grid,
    cv =5
)

In [16]:
knn_grid.fit(train_df[['Length','Diameter','Height','Whole_weight','Shucked_weight','Viscera_weight','Shell_weight','Rings']], train_df['Sex']
)

accuracies = pd.DataFrame(knn_grid.cv_results_)
accuracies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   mean_fit_time                            20 non-null     float64
 1   std_fit_time                             20 non-null     float64
 2   mean_score_time                          20 non-null     float64
 3   std_score_time                           20 non-null     float64
 4   param_kneighborsclassifier__n_neighbors  20 non-null     object 
 5   params                                   20 non-null     object 
 6   split0_test_score                        20 non-null     float64
 7   split1_test_score                        20 non-null     float64
 8   split2_test_score                        20 non-null     float64
 9   split3_test_score                        20 non-null     float64
 10  split4_test_score                        20 non-null

In [17]:
accuracies["sem_test_score"] = accuracies["std_test_score"] / 10**(1/2)
accuracies = (
    accuracies[[
        "param_kneighborsclassifier__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns={"param_kneighborsclassifier__n_neighbors": "n_neighbors"})
)
accuracies

n_neighbors  mean_test_score  sem_test_score
0            1         0.502822        0.000557
1            6         0.507518        0.005401
2           11         0.523523        0.008035
3           16         0.522593        0.009552
4           21         0.536700        0.009455
5           26         0.532931        0.008942
6           31         0.540457        0.004263
7           36         0.531988        0.005699
8           41         0.531985        0.005898
9           46         0.520698        0.002680
10          51         0.532460        0.004902
11          56         0.537636        0.003634
12          61         0.534815        0.003881
13          66         0.533878        0.005651
14          71         0.536226        0.005277
15          76         0.528701        0.003537
16          81         0.530581        0.002961
17          86         0.526818        0.002585
18          91         0.531987        0.002906
19          96         0.535283        0.002625

In [18]:
accuracy_vs_k = alt.Chart(accuracies).mark_line(point=True).encode(
    x=alt.X("n_neighbors").title("Neighbors"),
    y=alt.Y("mean_test_score")
        .scale(zero=False)
        .title("Accuracy estimate")
)

accuracy_vs_k

alt.Chart(...)

In [19]:
accuracies.iloc[accuracies['mean_test_score'].argmax()]

n_neighbors              31
mean_test_score    0.540457
sem_test_score     0.004263
Name: 6, dtype: object

In [20]:
# best accuracy score n_neighbors = 31

In [21]:
#Question 4
model_2 = KNeighborsClassifier(n_neighbors = 31)
knn_pipeline_2 = make_pipeline(preprocessor,model_2)

In [22]:
knn_pipeline_2.fit(train_df[['Length','Diameter','Height','Whole_weight',
                             'Shucked_weight','Viscera_weight','Shell_weight','Rings']], 
                   train_df['Sex'])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Length', 'Diameter',
                                                   'Height', 'Whole_weight',
                                                   'Shucked_weight',
                                                   'Viscera_weight',
                                                   'Shell_weight',
                                                   'Rings'])])),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=31))])

In [23]:
prediction = knn_pipeline_2.predict(test_df[['Length','Diameter','Height','Whole_weight',
                             'Shucked_weight','Viscera_weight','Shell_weight','Rings']])


In [29]:
test = pd.DataFrame({'Actual':test_df['Sex'],'Predicted':prediction})
pd.crosstab(
    test['Actual'],
    test['Predicted']
)

Predicted  Female  Male
Actual                 
Female        132   195
Male          118   264

In [25]:
from sklearn.metrics import recall_score, precision_score

accuracy = knn_pipeline_2.score(test_df[['Length','Diameter','Height','Whole_weight',
                             'Shucked_weight','Viscera_weight','Shell_weight','Rings']],
                    test_df['Sex'])

precision = precision_score(
    y_true = test_df['Sex'],
    y_pred = prediction,
    pos_label = 'Female'
)

recall= recall_score(
    y_true = test_df['Sex'],
    y_pred = prediction,
    pos_label = 'Female'
)

print(f'accuracy score : {accuracy}')
print(f'precision score: {precision}')
print(f'recall score: {recall}')

accuracy score : 0.5585331452750353
precision score: 0.528
recall score: 0.4036697247706422


In [26]:
#Question 5
from sklearn.compose import make_column_selector

accuracy_dict = {'size':[],'selected_predictors':[],'accuracy':[]}

names = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight',
       'Viscera_weight', 'Shell_weight', 'Rings']
n_totals = len(names)

selected = []

param_grid = {'kneighborsclassifier__n_neighbors': range(1,100,5)}
preprocessor = make_column_transformer(
    (StandardScaler(),make_column_selector(dtype_include='number'))
)

model = KNeighborsClassifier()
knn_pipeline_3 = make_pipeline(preprocessor,model)
knn_grid = GridSearchCV(
    estimator = knn_pipeline_3,
    param_grid = param_grid,
    cv = 5,
    n_jobs=-1
)

for i in range(1,n_totals+1):
    accs = np.zeros(len(names))
    for j in range(len(names)):
        X = train_df[selected + [names[j]]]
        y = train_df['Sex']

        knn_grid.fit(X,y)
        accuracies_grid = pd.DataFrame(knn_grid.cv_results_)
        accs[j] = accuracies_grid['mean_test_score'].max()
    best_set = selected + [names[accs.argmax()]]

    accuracy_dict['size'].append(i)
    accuracy_dict['selected_predictors'].append(','.join(best_set))
    accuracy_dict['accuracy'].append(accs.max())

    selected = best_set
    del names[accs.argmax()]

accuracies = pd.DataFrame(accuracy_dict)
accuracies
        

size                                selected_predictors  accuracy
0     1                                     Shucked_weight  0.549376
1     2                            Shucked_weight,Diameter  0.547988
2     3                     Shucked_weight,Diameter,Height  0.554560
3     4              Shucked_weight,Diameter,Height,Length  0.561615
4     5  Shucked_weight,Diameter,Height,Length,Shell_we...  0.552685
5     6  Shucked_weight,Diameter,Height,Length,Shell_we...  0.553611
6     7  Shucked_weight,Diameter,Height,Length,Shell_we...  0.553611
7     8  Shucked_weight,Diameter,Height,Length,Shell_we...  0.540457

In [27]:
accuracy_plot = alt.Chart(accuracies).mark_line(point=True).encode(
    x=alt.X('size').title('number of predictors'),
    y=alt.Y('accuracy').scale(zero=False)
)
accuracy_plot

alt.Chart(...)